In [4]:
import numpy as np
#import scipy.stats
#import seaborn as sns
import pandas as pd
import librosa
import librosa.display
from IPython.display import Audio, display
#import scikit_posthocs as sp
#import matplotlib.pyplot as plt
#from matplotlib import gridspec
#from matplotlib.transforms import BlendedGenericTransform
#import scikit_posthocs as sp
#from urllib.request import urlopen
#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

import os

import keras
import keras.layers as layers
from keras import backend as K
import numpy as np

Using TensorFlow backend.


TODO
Create data from clips
Create iterator to load data for all clips

Gener

In [16]:
# some test data to hack around with
test_file = "Assets\\DataShareArchive\\Test\\Noisy\\p232_010.wav"
wav, rate = librosa.core.load(test_file)
fft = librosa.stft(wav)

In [188]:
# Constants and settings
WINDOW_SIZE = 10
FFT_BINS = 1025

In [236]:
# Creating data from clip wave file
def clip_frames(file):
    wav, rate = librosa.core.load(file)
    half_win = WINDOW_SIZE//2
    fft = librosa.stft(wav) # organized as bins, frames
    frames = np.empty(shape=(0,FFT_BINS,WINDOW_SIZE))
    for i in range(0, fft.shape[1] - WINDOW_SIZE):
        frames = np.append(frames,[fft[:,i:i+WINDOW_SIZE]], axis=0)
    return frames


def clip_targets(file):
    wav, rate = librosa.core.load(file)
    half_win = WINDOW_SIZE//2
    fft = librosa.stft(wav)
    targets = np.empty(shape=(0,FFT_BINS,1))
    for i in range(half_win, fft.shape[1] - half_win):
        targets = np.append(targets, [fft[:,i+half_win:i+half_win+1]], axis=0)
    return targets

In [238]:
ft = clip_frames(test_file)
tt = clip_targets(test_file)

print (ft.shape, tt.shape)

(110, 1025, 10) (110, 1025, 1)


In [266]:
# Iterate over clean & noisy folders to create frames and targets
def create_data(root):
    clean_dir = root + "\\Clean\\"
    noisy_dir = root + "\\Noisy\\"
    frames = np.empty(shape=(0,FFT_BINS,WINDOW_SIZE))
    targets = np.empty(shape=(0,FFT_BINS,1))
    for file in os.listdir(clean_dir)[0:2]:
        filename = os.fsdecode(file)
        frames = np.concatenate((frames,clip_frames(noisy_dir + file)), axis=0)
        targets = np.concatenate((targets,clip_targets(clean_dir + file)), axis=0)
    return frames, targets


In [267]:
frames, targets = create_data("Assets\\DataShareArchive\\Test")

In [269]:
targets.shape

(172, 1025, 1)

In [84]:
# Define network

model = keras.Sequential(
     [layers.Dense(FFT_LEN, activation='relu', input_shape=[2]),
      layers.Dense(50, activation='relu'),
      layers.Dense(50, activation='relu'),
     layers.Dense(FFT_LEN, activation='linear')]
)

model.compile(optimizer='rmsprop', loss='mse')

In [80]:
type(targets[0])

numpy.ndarray

In [70]:
print(frames[0][5][0])

print(targets[0][0][0])

(-0.04013502-5.551115e-17j)
(0.16505659+1.3877788e-17j)


In [214]:
fft.shape
fft[:,1:2].shape

(1025, 1)

In [66]:
os.listdir("Assets\\DataShareArchive\\Test\\Noisy\\")[0:5]

['p232_001.wav',
 'p232_002.wav',
 'p232_003.wav',
 'p232_005.wav',
 'p232_006.wav']